Before you turn in the homework, make sure everything runs as expected. To do so, select **Kernel**$\rightarrow$**Restart & Run All** in the toolbar above.  Remember to submit both on **DataHub** and **Gradescope**.

Please fill in your name and include a list of your collaborators below.

In [ ]:
NAME = "Tianxiao Hu"
COLLABORATORS = ""

---

# Project 2: NYC Taxi Rides
# Part 2: EDA, Visualization, Feature Engineering

In this part, we will conduct EDA on the NYC Taxi dataset that we cleaned and train/validation split in part 1. We will also guide you through the engineering of some features that hopefully will help our model to accurately understand the data.


# Imports

Let us start by loading the Python libraries and custom tools we will use in this part.  

In [ ]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import os
from pathlib import Path

plt.rcParams['figure.figsize'] = (12, 9)
plt.rcParams['font.size'] = 12

sns.set(style="whitegrid", palette="muted")
%matplotlib inline

## Loading & Formatting data

The following code loads the data into a pandas DataFrame.

In [ ]:
# Run this cell to load the data. 
data_file = Path("data/part1", "cleaned_data.hdf")
train_df = pd.read_hdf(data_file, "train")

In [ ]:
train_df.head()

## 1: Data Overview 

As a reminder, the raw taxi data contains the following columns:
- `recordID`: primary key of this database
- `VendorID`: a code indicating the provider associated with the trip record
- `passenger_count`: the number of passengers in the vehicle (driver entered value)
- `trip_distance`: trip distance
- `tpep_dropoff_datetime`: date and time when the meter was engaged
- `tpep_pickup_datetime`: date and time when the meter was disengaged
- `pickup_longitude`: the longitude where the meter was engaged
- `pickup_latitude`: the latitude where the meter was engaged
- `dropoff_longitude`: the longitude where the meter was disengaged
- `dropoff_latitude`: the latitude where the meter was disengaged
- `duration`: duration of the trip in seconds
- `payment_type`: the payment type
- `fare_amount`: the time-and-distance fare calculated by the meter
- `extra`: miscellaneous extras and surcharges
- `mta_tax`: MTA tax that is automatically triggered based on the metered rate in use	
- `tip_amount`: the	amount of credit card tips, cash tips are not included
- `tolls_amount`: amount paid for tolls
- `improvement_surcharge`: fixed fee
- `total_amount`: total amount paid by passengers, cash tips are not included

Let us take a closer look at the target `duration` variable. In the cell below, we plot its distribution using `sns.distplot`. This should give us an idea about whether we have some outliers in our data.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

# Plot the distribution of duration using sns.distplot
# You can fill `ax=ax` to sns.distplot to plot in the ax object created above

sns.distplot(train_df['duration'], ax=ax, kde=False)

plt.title('Duration distribution');

As expected for a positive valued variable, we observe a skewed distribution. Note that we seem to have a handful of very long trips within our data. Use an appropriate data transformation to squeeze this highly-skewed distribution. Plot a `sns.distplot` of the transformed duration data for `train_df`.

In [ ]:
fig, ax = plt.subplots(figsize=(12, 9))

# Use a log transformation to squeeze the distribution
# You can add + 1 to all values before taking the log to handle possible 0 values for distribution

sns.distplot(np.log(train_df['duration'] + 1),
             ax=ax,
             axlabel='log(duration)',
             kde=False)

plt.title('log(duration) distribution');

After transforming our data, we should immediately observe that we are dealing with what seems to be log-normal distribution for the target variable `duration`. We can see the behavior of shorter rides better, whereas before they were lumped in a bar near 0. This is a nice result, since it can facilitate modeling later. 

**Note:** Keep in mind that we want to avoid peeking at our validation data because it may introduce bias. Therefore, we will be focusing on analyzing the training data for the remainder of this notebook.

## 2: Date Analysis

In order to understand the general pattern/trends of our taxi ride data, we will plot the number of taxi rides requested over time. Please run the following cell.

In [ ]:
plt.figure(figsize=(12, 9))

# Make a temporary copy of our datasets
tmp_train = train_df.copy()
tmp_train['date'] = tmp_train['tpep_pickup_datetime'].dt.date
tmp_train = tmp_train.groupby('date').count()['pickup_longitude']

# Plot the temporal overlap
plt.plot(tmp_train, '.-', label='train_df')

plt.title('Number of Taxi Rides per Start Day')
plt.xlabel("Start Date")
plt.legend()
plt.ylabel('Number of Taxi Rides');

### Question 2a
Taking a closer look at the plot above, we notice a drastic drop in taxi rides towards the end of Janurary.
What is the date corresponding to the lowest number of taxi rides? Enter your answer as a string in the format MM-DD-YYYY.

In [ ]:
lowest_rides_date = "01-23-2016"

# YOUR CODE HERE
# raise NotImplementedError()

print(lowest_rides_date)

In [ ]:
# Hidden test!

### Question 2b
What event could have caused this drop in taxi rides? Feel free to use Google.

In [ ]:
q2b_answer = r"""

THere is a crippling and historic blizzard that produced up to 3 ft (91 cm) of snow 
in parts of the Mid-Atlantic and Northeast United States. People choose to stay home
instead of going out by taxi. Thus there is a serious drop in taxi rides that day.

"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q2b_answer)

## 3. Spatial/Locational Analysis

We are curious about the distribution of taxi pickup/dropoff coordinates. We also may be interested in observing whether this distribution changes as we condition of longer/shorter taxi rides. In the cells below, we will categorize our data into long and short rides based on duration. Then we will plot the latitude and longitude coordinates of rides conditioned on these categories.

First you may want to familiarize yourself with a [map of Manhattan](https://www.google.com/maps/place/Manhattan,+New+York,+NY/@40.7590402,-74.0394431,12z/data=!3m1!4b1!4m5!3m4!1s0x89c2588f046ee661:0xa0b3281fcecc08c!8m2!3d40.7830603!4d-73.9712488).

Here we split `train_df` into two data frames, one called `short_rides` and one called `long_rides`. `short_rides` should contain all rides less than or equal to 15 minutes and `long_rides` should contain rides more than 15 minutes. 

**Note:** We chose 15 minutes because the mean duration of a ride is roughly 700 seconds ~ 12 minutes. We then round up to the nearest nice multiple of 5. Note that you should adjust how you determine short/long rides and outliers when feature engineering.

In [ ]:
short_rides = train_df[train_df["duration"] <= 900] # rides less than or equal to 15 mins
long_rides = train_df[train_df["duration"] > 900] # rides more than 15 minutes

In [ ]:
assert len(short_rides) == 12830
assert len(long_rides) == 5524

Below we generate 4 scatter plots. The scatter plots are ordered as follows:

* ax1: plot the __start__ location of short duration rides
* ax2: plot the __start__ location of long duration rides
* ax3: plot the __end__ location of short duration rides
* ax4: plot the __end__ location of long duration rides

In [ ]:
# Set random seed of reproducibility
random.seed(42)

# City boundaries
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)

# Define figure
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(ncols=2, nrows = 2, figsize=(16, 12), sharex=True, sharey=True)
alpha = 0.15 # make sure to include these as an argument
s = 1 # make sure to include this as an argument

short_rides.plot(kind = "scatter", x = "pickup_longitude", y = "pickup_latitude",
                     ax = ax1, alpha = alpha, s = s, title='Short Duration, Start Location')
long_rides.plot(kind = "scatter", x = "pickup_longitude", y = "pickup_latitude",
                    ax = ax2, alpha = alpha, s = s, title='Long Duration, Start Location')
short_rides.plot(kind = "scatter", x = "dropoff_longitude", y = "dropoff_latitude",
                     ax = ax3, alpha = alpha, s = s , title='Short Duration, End Location')
long_rides.plot(kind = "scatter", x = "dropoff_longitude", y = "dropoff_latitude",
                    ax = ax4, alpha = alpha, s = s, title='Long Duration, End Location')


fig.suptitle('Distribution of start/end locations across short/long rides.')


plt.ylim(city_lat_border)
plt.xlim(city_long_border);

### Question 3a

What do the plots above look like? 

In particular:
- Find what the following circled regions correspond to:
<img src="figs/nyc_locations.png" width="50%"></img>

**Hint: Here is a [page](https://www.google.com/maps/place/Manhattan,+New+York,+NY/@40.7590402,-74.0394431,12z/data=!3m1!4b1!4m5!3m4!1s0x89c2588f046ee661:0xa0b3281fcecc08c!8m2!3d40.7830603!4d-73.9712488) that may be useful.**

In [ ]:
q3a_answer = r"""
The big circle corresponds to Manhattan and small circle corresponds to LaGuardia Airport. The rectangular area
corresponds to Central Park.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q3a_answer)

### Question 3b
In each scatter plot above, why are there no points contained within the small rectangular region (towards the top left between the blue points)? Could this be an error/mistake in our data?

In [ ]:
q3b_answer = r"""
The rectangular region is Central Park. People are not allowed to drive there. It's not a error in our data.

"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q3b_answer)

### Question 3c
What observations/conclusions do you make based on the scatter plots above? In particular, how are trip duration and pickup/dropoff location related?

In [ ]:
q3c_answer = r"""
Many pick-up and drop-off locations are in Manhattan. LaGuardia Airport and John F. Kennedy International Airport 
are also hot places for taxi.
For short-distance taxi trip, the pick-up and drop-off locations are more located in Manhattan. However, 
for long distance taxi trip, many pick-up and drop-off points locates in two airports, which indicates that many
people choose to take a taxi to or from the airport.

"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q3c_answer)

This confirms that the trips are localized in NYC, with a very strong concentration in Manhattan **and** on the way to LaGuardia Airport. This might give you ideas of relevant features for feature engineering.  

Another way to visualize ride coordinates is using a **heat map** (this also helps us avoid overplotting). The following plots count the number of trips for NYC neighborhoods and areas, plotting with the `geopandas` package and theses [shapefiles](https://geo.nyu.edu/catalog/nyu_2451_36743) (do not mind the values on the colorbar). If you are curious about how to create the figures below, feel free to check out [`geopandas`](http://geopandas.org/).

![neighborhoods](figs/chloro_NY_neighborhoods.png)
![neighborhoods](figs/chloro_NY_taxi_zones.png)

## 4: Temporal features
We can utilize the `start_timestamp` column to design a lot of interesting features.

We implement the following temporal (related to time) features using the `add_time_columns` function below.
- `month` derived from `start_timestamp`.
- `week_of_year` derived from `start_timestamp`.
- `day_of_month` derived from `start_timestamp`.
- `day_of_week` derived from `start_timestamp`.
- `hour` derived from `start_timestamp`.
- `week_hour` derived from `start_timestamp`.

**Note 1**: You can use the `dt` attribute of the `start_timestamp` column to convert the entry into a `DateTime` object.

**Note 2**: We set `df.is_copy = False` to explicitly write back to the original dataframe, `df`, that is being passed into the `add_time_columns` function. Otherwise `pandas` will complain.

In [ ]:
def add_time_columns(df):
    """
    Add temporal features to df
    """
    df.is_copy = False 
    df.loc[:, 'month'] = df['tpep_pickup_datetime'].dt.month
    df.loc[:, 'week_of_year'] = df['tpep_pickup_datetime'].dt.weekofyear
    df.loc[:, 'day_of_month'] = df['tpep_pickup_datetime'].dt.day
    df.loc[:, 'day_of_week'] = df['tpep_pickup_datetime'].dt.dayofweek
    df.loc[:, 'hour'] = df['tpep_pickup_datetime'].dt.hour
    df.loc[:, 'week_hour'] = df['tpep_pickup_datetime'].dt.weekday * 24 + df['hour']
 
    # No real need to return here, but we harmonize with remove_outliers for later pipelinezation
    return df

In [ ]:
# Note that we are applying this transformation to train_df, short_rides and long_rides
train_df = add_time_columns(train_df)
short_rides = add_time_columns(short_rides)
long_rides = add_time_columns(long_rides)

In [ ]:
train_df[['month', 'week_of_year', 'day_of_month', 'day_of_week', 'hour', 'week_hour']].head()

Your `train_df.head()` should look like this, although the ordering of the data in `id` might be different:
![time_columns](figs/time_columns.png)

In [ ]:
time_columns = ['month',
                'week_of_year',
                'day_of_month',
                'day_of_week',
                'hour',
                'week_hour']


# Check columns were created
assert all(column in train_df.columns for column in time_columns)

# Check type
assert train_df[time_columns].dtypes.nunique() == 1

assert train_df[time_columns].dtypes.nunique() == 1

### Visualizing Temporal Features

### Question 4a
Let us now use the features we created to plot some histograms and visualize patterns in our dataset. We will analyze the distribution of the number of taxi rides across months and days of the week. This can help us visualize and understand patterns and trends within our data.

This is a open ended question. Create 2 plots that visualize temporal information from our dataset. At least one of them must visualize the hour of each day. Aside from that you can use any column from `time_columns`. 

You can use the same column multiple times, but if the plots are redundant you will not receive full credit. This will be graded based on how informative each plot is and how "good" the visualization is (remember what good/bad visualizations look like for different kinds of data!).  

#### Visualization 1

In [ ]:
# Visualization 1
# YOUR CODE HERE
# raise NotImplementedError()
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
plt.subplot(121)
sns.distplot(short_rides["hour"], bins=24, color="green")
plt.title("Short rides distribution of a day")
plt.subplot(122)
sns.distplot(long_rides["hour"], bins=24)
plt.title("Long rides distribution of a day")

#### Visualization 2

In [ ]:
# Visualization 2
# YOUR CODE HERE
# raise NotImplementedError()
fig, axs = plt.subplots(1, 2, figsize=(9, 3))
plt.subplot(121)
sns.distplot(short_rides["day_of_week"], bins=7, color="green", kde=False)
plt.title("Short rides distribution of a week")
plt.subplot(122)
sns.distplot(long_rides["day_of_week"], bins=7, kde=False)
plt.title("Long rides distribution of a week")

### Question 4b
Briefly explain for each plot
1. What feature you're visualization
2. Why you chose this feature
3. Why you chose this visualization method

In [ ]:
q4b_answer = r"""
For the first plot:
I choose to visusalize the `hour` feature. Try to show the distribution of the number of taxi rides 
across hours during the day. I choose this feature mainly for see if taxi drivers will be more busy
during rush hours. I use a barplot to show each hour's number of taxi rides, for the x-axis hour is 
discrete and the number of taxi riders are continuous.

For the second plot:
I choose to visusalize the `day_of_week` feature. Try to show the distribution of the number of taxi rides 
across days during the whole week. I choose this feature mainly for see if taxi drivers will be more busy
during workdays. I use a barplot to show each day's number of taxi rides, for the x-axis day of the week is 
discrete and the number of taxi riders are continuous.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q4b_answer)

### Question 4c

From the various plots above, what conclusions can you draw about the temporal aspects of our data? How does this relate to duration? 

In [ ]:
q4c_answer = r"""
There will be less taxi rides from midnight to dawn. More taxi rides happen during the day, and when people
leave work and go home(during 6pm - 8pm) there are more taxi rides. There are not significant difference 
between short rides and long rides.

There are more short taxi rides during Friday to Sunday, which indicates more people take taxi to play and 
have fun. However, when it comes to long taxi rides, more of them happen during Tuesday to Friday. We can
assume that people take taxi for long-distance transportation during workdays.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q4c_answer)

### Question 4d

Previously, we have analyzed the temporal features `hour` and `day_of_week` independently, but these features may in fact have a relationship between each other. Determining the extent to their relationship may be useful in helping us create new features in our model. Create a violin plot that displays distribution of rides over each hour per day of the week.

In [ ]:
fig, axes = plt.subplots(1, 1, figsize=(10, 8))
days_of_week = ["Monday", "Tuesday", "Wednesday", "Thursday", "Friday", "Saturday", "Sunday"]


# YOUR CODE HERE
# raise NotImplementedError()
sns.violinplot(x="day_of_week", y="hour", data=train_df)
plt.xlabel('Day of the Week')
plt.ylabel('Hour of the Day')
plt.xticks(list(range(7)), days_of_week) 
plt.title('Distribution of the Number of Taxi Rides over each Hours by Day of th e Week')
# raise NotImplementedError()

plt.tight_layout();

### Question 4e
Do you notice anything interesting about your visualization? How would you explain this plot to a lay person? What are the features/patterns of interest?

In [ ]:
q4e_answer = r"""
At the first glance the violin shapes look quite same to each other. However, we notice that 
there are more taxi rides during the morning in Friday, Saturday and Sunday.
We can explain to a lay person like this. This violin graph visualize the distribution of days
through a week, each violin corresponds to a day and the width of violin represents the distribution
during the day. 
In each day the taxi rides starting from 5am to 8pm occupy the majority of a day. There are more 
people take taxi during midnight in weekends.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q4e_answer)

## 5: Vendors

Recall that in Part 1, we found that there are only two unique vendors represented in the dataset. We may wonder if the vendor feature can be useful when trying to understand taxi ride duration.

### Question 5a
Visualize the VendorID feature. Create at least one plot that gives insight as to whether this feature would be useful or not in our model.

In [ ]:
# Visualization

# YOUR CODE HERE
# raise NotImplementedError()

vendor_1_short_hour = train_df[(train_df['VendorID'] == 1) & (train_df["duration"] <= 900)].groupby('hour').count().reset_index()
vendor_1_long_hour = train_df[(train_df['VendorID'] == 1) & (train_df["duration"] > 900)].groupby('hour').count().reset_index()
vendor_2_short_hour = train_df[(train_df['VendorID'] == 2) & (train_df["duration"] <= 900)].groupby('hour').count().reset_index()
vendor_2_long_hour = train_df[(train_df['VendorID'] == 2) & (train_df["duration"] > 900)].groupby('hour').count().reset_index()
sns.lineplot(x="hour", y="month", data=vendor_1_short_hour, label="vendor 1 short")
sns.lineplot(x="hour", y="month", data=vendor_2_short_hour, label="vendor 2 short")
sns.lineplot(x="hour", y="month", data=vendor_1_long_hour, label="vendor 1 long")
sns.lineplot(x="hour", y="month", data=vendor_2_long_hour, label="vendor 2 long")
plt.title("Distribution of the Number of Taxi Rides of Different Vendor by Hour")
plt.xlabel("Hour of the Day")
plt.ylabel("Number of Taxi Rides")

### Question 5b
Justify why you chose this visualization method and how it helps determine whether `vendor_id` is useful in our model or not.

In [ ]:
q5b_answer = r"""
The x-axis is `hour of the day` and discrete, the y-axis is number of taxi rides and is continuous. 
Using a line graph can visualize the trends of number of taxi rides during a day and we can use different 
color of lines to distinguish vendors. We can see the in rush hours, vendor 2 have more taxi rides, both 
short and long rides.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q5b_answer)

### Question 5c
From the plot above, do you think vendor_id will help us understand duration? Why or why not?

In [ ]:
q5c_answer = r"""
We can see from the plot above vendor_id have some influence on the number of taxi rides. However,
the difference is not very significant, so it's a bit hard for us to identify the difference only
by vendor_id.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q5c_answer)

## 6: Distance features

We can also use the coordinates information to compute distance features.  This will allow us to compute speed related features.  
We will compute the [haversine](https://en.wikipedia.org/wiki/Haversine_formula) distance, the [manhattan](https://en.wikipedia.org/wiki/Taxicab_geometry) distance and the [bearing](http://www.mathsteacher.com.au/year7/ch08_angles/07_bear/bearing.htm) angle.

In [ ]:
# These functions are implemented for you
def haversine(lat1, lng1, lat2, lng2):
    """
    Compute haversine distance
    
    The haversine formula determines the great-circle distance between two points 
    on a sphere given their longitudes and latitudes. Important in navigation, it 
    is a special case of a more general formula in spherical trigonometry, 
    the law of haversines, that relates the sides and angles of spherical triangles.
    """
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    average_earth_radius = 6371
    lat = lat2 - lat1
    lng = lng2 - lng1
    d = np.sin(lat * 0.5) ** 2 + np.cos(lat1) * np.cos(lat2) * np.sin(lng * 0.5) ** 2
    h = 2 * average_earth_radius * np.arcsin(np.sqrt(d))
    return h

def manhattan_distance(lat1, lng1, lat2, lng2):
    """
    Computes Manhattan distance
    
    The name alludes to the grid layout of most streets on the island of Manhattan, 
    which causes the shortest path a car could take between two intersections in the borough 
    to have length equal to the intersections' distance in taxicab geometry.
    """
    a = haversine(lat1, lng1, lat1, lng2)
    b = haversine(lat1, lng1, lat2, lng1)
    return a + b

def bearing(lat1, lng1, lat2, lng2):
    """
    Compute the bearing, or angle, from (lat1, lng1) to (lat2, lng2).
    A bearing of 0 refers to a NORTH orientation.
    """
    lng_delta_rad = np.radians(lng2 - lng1)
    lat1, lng1, lat2, lng2 = map(np.radians, (lat1, lng1, lat2, lng2))
    y = np.sin(lng_delta_rad) * np.cos(lat2)
    x = np.cos(lat1) * np.sin(lat2) - np.sin(lat1) * np.cos(lat2) * np.cos(lng_delta_rad)
    return np.degrees(np.arctan2(y, x))

In [ ]:
def add_distance_columns(df):
    df.loc[:, 'manhattan'] = manhattan_distance(lat1=df['pickup_latitude'],
                                                lng1=df['pickup_longitude'],
                                                lat2=df['dropoff_latitude'],
                                                lng2=df['dropoff_longitude'])

    df.loc[:, 'bearing'] = bearing(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    df.loc[:, 'haversine'] = haversine(lat1=df['pickup_latitude'],
                                   lng1=df['pickup_longitude'],
                                   lat2=df['dropoff_latitude'],
                                   lng2=df['dropoff_longitude'])
    
    return df

In [ ]:
train_df = add_distance_columns(train_df)
short_rides = add_distance_columns(short_rides)
long_rides = add_distance_columns(long_rides)

In [ ]:
fig, axes = plt.subplots(4, 1, figsize=(12, 9))
sns.distplot(train_df['haversine'], ax=axes[0], axlabel='haversine');
sns.distplot(train_df['manhattan'], ax=axes[1], axlabel='manhattan');
sns.distplot(train_df['bearing'], ax=axes[2], axlabel='bearing');
sns.distplot(train_df['trip_distance'], ax=axes[3], axlabel='trip_distance');

sns.despine(left=True);
plt.setp(axes, yticks=[]);
plt.tight_layout();


### Question 6a

The `bearing` direction is angle, the initial direction of the trip.  
The bearing direction has two prominent peaks around 30 and -150 degrees.  
**Can you relate these peaks to the orientation of Manhattan? What do you notice about these angles?**

**Hint:** This [wikipedia article](https://en.wikipedia.org/wiki/Commissioners%27_Plan_of_1811) has the answer, although it may take some digging. Alternatively, try to look at a map of Manhattan.

In [ ]:
q6a_answer = r"""
The shape of Manhattan is from 30 to -150 degrees. So there is high probability 
that the initial direction of the trip is along the direction which Manhattan is 
stretching. 
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q6a_answer)

### Question 6b

For haversine and manhattan distances, it is probably more helpful to look at the log distribution. We are also curious about whether these distance features can help us understand duration. Create at least one plot that compares haversine and manhattan distances and gives insight as to whether this would be a useful feature in our model.

In [ ]:
train_df["log_haversine"] = np.log(train_df["haversine"])
train_df["log_manhattan"] = np.log(train_df["manhattan"])

In [ ]:
train_df_dropna = train_df[np.logical_and(np.isfinite(train_df["log_haversine"]), np.isfinite(train_df["log_manhattan"]))]

In [ ]:
sns.jointplot(data=train_df_dropna, x="log_haversine", y="log_manhattan", kind="reg", dropna=True)
plt.title("Relationship between `log_haversine` and `log_manhattan`")

In [ ]:
sns.jointplot(data=train_df_dropna, x="log_haversine", y="duration", kind="reg", dropna=True)
plt.title("Relationship between `log_haversine` and `duration`")

In [ ]:
sns.jointplot(data=train_df_dropna, x="log_manhattan", y="duration", kind="reg", dropna=True)
plt.title("Relationship between `log_manhattan` and `duration`")

In [ ]:
# Visualization
# YOUR CODE HERE
# raise NotImplementedError()

### Question 6c
Justify why you chose this visualization method and how it helps inform you about using manhattan/haversine distance as a feature for predicting trip duration.

In [ ]:
q6c_answer = r"""
I choose jointplot to visualize the relationship between log distribution of manhattan, 
log distribution of haversine and duration. They are all continuous variables jointplot
is good method for visualize the relationships between continuous variables. I find the
distribution of log_manhattan and log_haversine is nearly normal distribution and they 
both have positive relation with duration.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q6c_answer)

### Question 6d
Fill in the code below to plot a scatter plot of manhattan distance vs duration.

In [ ]:
# YOUR CODE HERE
# raise NotImplementedError()
sns.scatterplot(data=train_df, x="manhattan", y="duration")
plt.title("Relationship between `manhattan distance` vs. `duration`")

### Question 6e

According to the plot above, there are a few outliers in both duration and manhattan distance. **Which type of outliers is most likely to be a mistake in our data?**

In [ ]:
q6e_answer = r"""
The outliers with small manhattan distance and long duration are most likely to be a mistake.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q6e_answer)

## 7: Advanced features
You do not need to incorporate these features into your model, although it may help lower your error. You are required to read through this portion and respond to the questions. All of the code is provided, please skim through it and try to understand what each cell is doing.

### Clustering
[Clustering](https://en.wikipedia.org/wiki/Cluster_analysis) is the task of grouping objects such that members within each group are more similar to each other than members of other groups. Clustering is a powerful tool used in many fields, including machine learning, pattern recognition, image analysis, information retrieval, bioinformatics, data compression, and computer graphics. Recall cluster sampling, which we learned earlier in the semester. We will use a simple clustering method (clustering by spatial locality) to reveal some more advanced features.

### Speed features

For `train_df`, we have the `duration` and now some distance information.  
This is enough for us to compute average speed and try to better understand our data.   

For `test_df`, we cannot use `duration` as a feature because it is what we are trying to predict. One clever way to include speed information for modeling would be as follows:

1. Cluster the observations in `train_df` by rounding the latitude and longitudes.
2. Compute the average speed per pickup cluster and dropoff cluster.
3. Match each observation in `test_df` to its pickup cluster and dropoff cluster based off the latitude and longitude, thus assigning the average speed for the pickup and dropoff cluster.
4. We have added speed information as features for `test_df`.

Therefore, we have propagated information computed in the `train_df` into the `test_df` via clustering. This is not something we will do in this notebook, although you can try it for yourself!

Other information that could be added based on clustering (both pickup cluster and dropoff cluster):
- Average of `avg_speed_h` per cluster.
- Average of `duration` per cluster.
- Average of `avg_speed_h` per cluster and hour.
- Average of `duration` per cluster and hour.
- In-cluster flow of trips for 60 min period.
- Out-cluster flow of trips for 60 min period.

In [ ]:
# Calculate average manhattan speed
train_df['avg_speed_m'] = 1000 * train_df['manhattan'] / train_df['duration']
train_df['avg_speed_m'] = train_df['avg_speed_m'][train_df['avg_speed_m'] < 100]
train_df['avg_speed_m'].fillna(train_df['avg_speed_m'].median(), inplace=True)

In [ ]:
train_df['avg_speed_m'].describe()

In [ ]:
# Visualize average manhattan speed by hour, day of week and week hour
fig, axes = plt.subplots(ncols=3, figsize=(15, 6), sharey=True)

axes[0].plot(train_df.groupby('hour').mean()['avg_speed_m'], 'bo-', lw=1, alpha=0.7)
axes[1].plot(train_df.groupby('day_of_week').mean()['avg_speed_m'], 'go-', lw=1, alpha=0.7)
axes[2].plot(train_df.groupby('week_hour').mean()['avg_speed_m'], 'ro-', lw=1, alpha=0.7)

axes[0].set_xlabel('hour')
axes[1].set_xlabel('day_of_week')
axes[2].set_xlabel('week_hour')
axes[0].set_ylabel('Manhattan average speed');

### Question 7a
Based off of these visualizations, provide 2-3 insights on the average speed.

In [ ]:
q7a_answer = r"""
1. The Manhattan average speed reaches the highest point in 5 a.m. for the traffic is light. However, 
it drops sharply after 8 a.m. when people begin to go to work. It's common sense that when there is 
light traffic drivers can drive faster but when there is heavy traffic it will be much more slower.

2. The Manhattan average speed is smaller during workdays than weekends. That's because in weekends people
will not take taxi rides during rush hours and the traffic is more controlable, leading to higher Manhattan
average speed.

3. The Manhattan average speed goes up and down periodically, corresponding to each day in the week. 
In the morning the average speed will go up and in rush hours it will go down. 
"""

print(q7a_answer)

We are now going to visualize the average speed per region. Here we define regions as a very basic classical clustering based on rounding of spatial coordinates.

In [ ]:
# Round / bin the latitude and longitudes
train_df['start_lat_bin'] = np.round(train_df['pickup_latitude'], 3)
train_df['start_lng_bin'] = np.round(train_df['pickup_longitude'], 3)

# Average speed for regions
gby_cols = ['start_lat_bin', 'start_lng_bin']

coord_stats = (train_df.groupby(gby_cols)
               .agg({'avg_speed_m': 'mean', 'manhattan': 'count'})
               .reset_index())

coord_stats = coord_stats[coord_stats['manhattan'] > 10]

In [ ]:
# Visualize the average speed per region
city_long_border = (-74.03, -73.75)
city_lat_border = (40.63, 40.85)
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(14, 10))

scatter_trips = ax.scatter(train_df['pickup_longitude'].values,
                           train_df['pickup_latitude'].values,
                           color='grey', s=1, alpha=0.5)

scatter_cmap = ax.scatter(coord_stats['start_lng_bin'].values,
                          coord_stats['start_lat_bin'].values,
                          c=coord_stats['avg_speed_m'].values,
                          cmap='viridis', s=10, alpha=0.9)

cbar = fig.colorbar(scatter_cmap)
cbar.set_label("Manhattan average speed")
ax.set_xlim(city_long_border)
ax.set_ylim(city_lat_border)
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
plt.title('Heatmap of Manhattan average speed')
plt.axis('off');

### Question 7b
In 2-3 sentences, describe how we can use the clustering visualization above to gain insight on the speed. Do you think spatial clustering would be useful in reducing the error of our model?

In [ ]:
q7b_answer = r"""
Places near two airports has a lighter color, indicating that Manhattan average speed there is higher there.
In Manhattan the average speed goes more lower due to the heavy traffic. 
Spatial clustering would be useful in reducing the error of our model for we can extract the general pattern 
or features of that clusters and leaving out outliers. However we must carefully choose the granularity of 
spatial clustering. Only by choosing appropriate cluster numbers can we capture important pattern in our data.
"""

# YOUR CODE HERE
# raise NotImplementedError()

print(q7b_answer)

## Part 2 Exports
We are not requiring you to export anything from this notebook, but you may find it useful to do so. There is a space below for you to export anything you wish.

In [ ]:
Path("data/part2").mkdir(parents=True, exist_ok=True)
data_file = Path("data/part2", "data_part2.hdf") # Path of hdf file
...

## Part 2 Conclusions 

We now have a good understanding of the taxi data we are working with.
Visualizing large amounts of data can be a difficult task. One helpful tool is [datashader](https://github.com/bokeh/datashader), a data rasterization pipeline for automating the process of creating meaningful representations of large amounts of data. Using the [geopandas](http://geopandas.org/) package also makes working with geospatial data easier. We encourage you to explore these tools if you are interested in learning more about visualization!

Within our taxi data set, we have explored different features and their relationship with ride duration. Now, we are ready to incorporate more data in order to add to our set of features.

**Please proceed to part 3 where we will be engineering more features and building our models using a processing pipeline.**

## Submission

You're almost done!

Before submitting this assignment, ensure that you have:

1. Restarted the Kernel (in the menubar, select Kernel$\rightarrow$Restart & Run All)
2. Validated the notebook by clicking the "Validate" button.

Then,

1. **Submit** the assignment via the Assignments tab in **Datahub** 
1. **Upload and tag** the manually reviewed portions of the assignment on **Gradescope**